In [1]:
import pandas as pd
from io import StringIO
import csv
import re
from collections import defaultdict
import numpy as np
import math

In [3]:
types = defaultdict(str, phone="str")

In [4]:
dfs = [None] * 3
csvs = ["facebook_dataset.csv", "google_dataset.csv", "website_dataset.csv"]
seps = [",", ",", ";"]

def clean_text(text):
    return re.sub('([^,\n])"{2,3}', '\1"', text)

for i, csv_name in enumerate(csvs):
    with open(csv_name, 'r') as file:
        # each csv contains \", so we have to save it
        data = file.read().replace('\\"', 'PLACEHOLDER_FOR_QUOTE')
        # these cleanup is needed because of website_dataset
        data = clean_text(data)
    csv_data = StringIO(data)

    dfs[i] = pd.read_csv(csv_data, sep=seps[i], dtype=types)
    dfs[i].replace('PLACEHOLDER_FOR_QUOTE', "\"", inplace=True)
    print("Reading " + csv_name + "... done")

Reading facebook_dataset.csv... done
Reading google_dataset.csv... done
Reading website_dataset.csv... done


In [5]:
facebook_df = dfs[0]
google_df = dfs[1]
website_df = dfs[2]
print("facebook csv info:\n")
facebook_df.info()
print("\ngoogle csv info:\n")
google_df.info()
print("\nwebsite csv info:\n")
website_df.info();


facebook csv info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72010 entries, 0 to 72009
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   domain              72010 non-null  object
 1   address             57380 non-null  object
 2   categories          55310 non-null  object
 3   city                45106 non-null  object
 4   country_code        57874 non-null  object
 5   country_name        45404 non-null  object
 6   description         28003 non-null  object
 7   email               20289 non-null  object
 8   link                72010 non-null  object
 9   name                72009 non-null  object
 10  page_type           72004 non-null  object
 11  phone               44866 non-null  object
 12  phone_country_code  38013 non-null  object
 13  region_code         45101 non-null  object
 14  region_name         45101 non-null  object
 15  zip_code            36049 non-null  object
dtypes:

The columns that interest us the most are Category, Address (country, region...), Phone, Company names.

In [54]:
cat_f = facebook_df["categories"].drop_duplicates()
print("Facebook company unique elements:\n")
print(cat_f)
cat_g = google_df["category"].drop_duplicates()
print("\nGoogle company unique elements:\n")
print(cat_g)
cat_w = website_df["s_category"].drop_duplicates()
print("\nWebsite company unique elements:\n")
print(cat_w)

Facebook company unique elements:

0                                                      NaN
1        Appliance Repair & Maintenance|Home Builders &...
2              Boats & Yachts Dealers|Boat Tours & Cruises
3        Architects & Architectural Services|Other Engi...
5        Other schools|High Schools|Community Center|Ad...
                               ...                        
71915    Auto Services|Auto Parts Store|Sporting Goods ...
71923    Parts & Accessories|Digital & Marketing Agenci...
71929    Community Center|CPR Training|Adult Education ...
71946    Security Guards & Patrol Services|Social Servi...
71953    Garden Equipment & Supplies|Landscaping & Lawn...
Name: categories, Length: 10185, dtype: object

Google company unique elements:

0                   Fabric-Based Home Goods
1                               Book Stores
2         Other Building Material Retailers
3                   Plastic Surgery Clinics
4                       Catering & Delivery
               

Facebook csv has a different parser on categories, for that we will create a new list for finding out the categories from it:

In [80]:
facebook_cat_list = set([a for b in cat_f.str.split('|').tolist()[1:] for a in b])
len(facebook_cat_list)

406

In [82]:
# categories from google minus categories from website
a = [x for x in cat_g if x not in cat_w]
print(len(a), len(cat_g), len(cat_w) - len(cat_g))
a = [x for x in facebook_cat_list if x not in cat_w]
print(len(a), len(facebook_cat_list), len(cat_w) - len(facebook_cat_list))
a = [x for x in facebook_cat_list if x not in cat_g]
print(len(a), len(facebook_cat_list), len(cat_g) - len(facebook_cat_list))

471 92
406 157
406 65


From the substractions of the categories list, we understand that:
 - google csv and facebook csv have less categories than website csv, but at least all the categories from google and facebook csv part can be located in the categories of teh website csv 
 - same for facebook csv and google csv, whereas google csv has more categories than facebook csv, but still facebook csv categories can be found in all of the other csvs
 - the intersection between all of the categories is facebook csv categories, and most of them are found in website csv

Next, we will analyse the domain of each company:

In [6]:
dom_f = facebook_df["domain"]
print("Facebook company domains:\n")
print(len(facebook_df), len(dom_f), len(dom_f.drop_duplicates()))
dom_g = google_df["domain"]
print("\nGoogle company domains:\n")
print(len(google_df), len(dom_g), len(dom_g.drop_duplicates()))
dom_w = website_df["root_domain"]
print("\nWebsite company domains:\n")
print(len(website_df), len(dom_w), len(dom_w.drop_duplicates()))

Facebook company domains:

72010 72010 72010

Google company domains:

356520 356520 72010

Website company domains:

72018 72018 72018


In [168]:
len(dom_g[dom_g.duplicated()].unique())

15291

Facebook and Website csvs are fine, but Google csv has a lot of websites in which the domain is instagram or other well known websites. In total, there are 15000 websites which are used more than once.

In [162]:
dom_w_to_list = dom_w.tolist()
dom_g_to_list = dom_g.tolist()
dom_f_to_list = dom_f.tolist()
a = [x for x in dom_f_to_list if x not in dom_w_to_list]
print(len(a), len(dom_f) - len(dom_w))

a = [x for x in dom_f_to_list if x not in dom_g_to_list]
print(len(a), len(dom_f) - len(dom_g.drop_duplicates()))

a = [x for x in dom_g_to_list if x not in dom_f_to_list]
print(len(a), len(dom_f) - len(dom_g.drop_duplicates()))

a = [x for x in dom_w_to_list if x not in dom_f_to_list]
print(len(a), a)

5 -8
0 0
0 0
13 ['MARTIN-LAFLAMM\x01', 'Funeral Hom\x01', ' Mobile Pet Salon - Edmonto\x01', 'YACHTS IN\x01', 'Fitzwilliam NH 03447', '603.585.300\x01', 'Avenida Food Hall & Fresh Marke\x01', 'Paint & ', 'Decor Inc\x01', nan, '(519) 771-850\x01', "Let's Be Crystal Clea\x01", 'Â\x01']


So it seems that all the Facebook and Google domains (only unique) are the same. Website csv brings 13 different domains in addition, which all seems to be wrong.

Lets take into consideration as well the name of each company, as Google csv does not have unique domains.

In [101]:
len(google_df[google_df["name"].isna()])

31

In [144]:
name_f = facebook_df["name"].dropna()
print("Facebook company domains:\n")
print(len(facebook_df), len(name_f), len(name_f.drop_duplicates()))
name_g = google_df["name"].dropna()
print("\nGoogle company domains:\n")
print(len(google_df), len(name_g), len(name_g.drop_duplicates()))
name_w = website_df["site_name"].dropna()
print("\nWebsite company domains:\n")
print(len(website_df), len(name_w), len(name_w.drop_duplicates()))

Facebook company domains:

72010 72009 71837

Google company domains:

356520 356489 353898

Website company domains:

72018 68714 67541


In [161]:
name_f[name_f.duplicated()]

4756                               Chorus
7504                         Clean Freaks
7611     Ministerio Público de la Defensa
7810                           Volkswagen
9882                                  JLL
                       ...               
71006                       Grace Academy
71330                              Nestlé
71331                Methodist Day School
71492                       City of Perth
71716                            Michelin
Name: name, Length: 172, dtype: object

In [145]:
name_f_to_list = name_f.tolist()
name_g_to_list = name_g.tolist()
name_w_to_list = name_w.tolist()
a = [x for x in name_f_to_list if x not in name_w_to_list]
print(len(a), len(name_f) - len(name_w))

a = [x for x in name_f_to_list if x not in name_g_to_list]
print(len(a), len(name_f) - len(name_g))

a = [x for x in name_w_to_list if x not in name_f_to_list]
print(len(a), len(name_w) - len(name_f))

47967 3295
40527 -284480
44540 -3295


For the name column of each csv, we can observe more duplicates, and also some of them 

Now, for the last part which could be a better representation for uniqueness of a company: phone number 

In [17]:
phone_number_f = facebook_df["phone"].dropna()
print("Facebook company phone numbers:\n")
print(len(facebook_df), len(phone_number_f), len(phone_number_f.drop_duplicates()))
phone_number_g = google_df["phone"].dropna()
print("\nGoogle company numbers:\n")
print(len(google_df), len(phone_number_g), len(phone_number_g.drop_duplicates()))
phone_number_w = website_df["phone"].dropna()
print("\nWebsite company numbers:\n")
print(len(website_df), len(phone_number_w), len(phone_number_w.drop_duplicates()))

Facebook company phone numbers:

72010 44866 44787

Google company numbers:

356520 323948 285821

Website company numbers:

72018 65640 60053


In [18]:
facebook_df[facebook_df["phone"].duplicated() & facebook_df["phone"].notna()]["phone"]

4273     +18009525210
5328     +14032735683
12420    +17809106688
12614    +19056909209
13835    +18173294161
             ...     
66486    +15167466585
68226    +17783981111
69436    +17057894345
70294    +14033200383
71278    +14506803287
Name: phone, Length: 79, dtype: object

In [19]:
def remove_plus_spaces_lines(text):
    if pd.isna(text):
        return text
    return text.replace("+", "").replace(" ", "").replace("-", "").replace('(', "").replace(')', "")

f = phone_number_f.drop_duplicates().map(remove_plus_spaces_lines).tolist()
w = phone_number_w.drop_duplicates().map(remove_plus_spaces_lines).tolist()
g = phone_number_g.drop_duplicates().map(remove_plus_spaces_lines).tolist()
a = [x for x in w if x not in f]
print(len(a))
a = [x for x in f if x not in g]
print(len(a))
a = [x for x in w if x not in g]
print(len(a))

29458
6838
15765


It seems that the phone numbers differ a lot, except facebook - google, or maybe there is a problem from their format and removing spaces, lines, paranthesis and the + from the beginning is not enough

After receiving some good details about each column, now we can finally have an approach on the problem:
Unfortunately, this problem cannot be solved with a simple "join" between the columns, as we have to check the other details for each company.
For the beginning, I consider that the domain column, even if it is not wanted, it is really a great approach for the beginning, due to no duplicates except google and we can finish really fast with 70 000 data. We will prefer the category of facebook, because it is a much better representation of the data. For the beginnning, we will add only the ones from the intersection between google and facebook. In addition, we will add a frequency array for choosing which attribute is for the best if there are conflicts in names, phone numbers etc. If frequeencies are equal, we will check which one is more unique.

For the other 13 domains from website csv, they won't be used because they look a bit corrupted, and from my perspective, I should not work with them until I talk with someone about them

In [7]:
duplicated_dom = dom_g[dom_g.duplicated()].unique().tolist()

In [50]:
new_df = pd.DataFrame(columns=["Category", "Country", "Region", "Phone Number", "Name", "Domain"])
dummy_website_df = pd.DataFrame([[pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]], columns=["s_category", "main_country", "main_region", "phone", "site_name", "root_domain"])
dummy_google_df = pd.DataFrame([[pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]], columns=["category", "country_name", "region_name", "phone", "name", "domain"])
dummy_facebook_df = pd.DataFrame([[pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]], columns=["categories", "country_name", "region_name", "phone", "name", "domain"])
# we will need all of the repeated domains to make sure that the companies from facebook and website csv are the same as the ones from google

def create_freq_dict(row1, row2, row3, key1, key2, key3):
    my_dict = {}    
    if pd.notna(row1[key1]):
        my_dict[row1[key1].lower()] = my_dict.get(row1[key1].lower(), 0) + 1
    if pd.notna(row2[key2]):
        my_dict[row2[key2].lower()] = my_dict.get(row2[key2].lower(), 0) + 1
    if pd.notna(row3[key3]):
        my_dict[row3[key3].lower()] = my_dict.get(row3[key3].lower(), 0) + 1

    return my_dict

for el in dom_f:
    website_row = website_df[website_df["root_domain"] == str(el)]
    facebook_row = facebook_df[facebook_df["domain"] == str(el)]
    google_row = google_df[google_df["domain"] == str(el)]
    if len(website_row) == 0:
        website_row = dummy_website_df
    if len(google_row) == 0:
        google_row = dummy_google_df
    new_row = [""] * 5 + [str(el)]

    if str(el) in duplicated_dom:
        continue
    else:
        # for category
        facebook_row = facebook_row.iloc[0]
        google_row = google_row.iloc[0]
        website_row = website_row.iloc[0]
        new_row[0] = str(facebook_row["categories"]) \
            if pd.notna(facebook_row["categories"]) else str(website_row["s_category"]) \
            if pd.notna(website_row["s_category"]) else str(google_row["category"])
        if pd.isna(website_row["s_category"]) and str(website_row["s_category"]) not in new_row[0]:
            new_row[0] += "|" + str(website_row["s_category"])
        if pd.notna(google_row["category"]) and str(google_row["category"]) not in new_row[0]:
            new_row[0] += "|" + str(google_row["category"])
        # for country, region, phone number and name
        country_freq = create_freq_dict(website_row, facebook_row, google_row, "main_country", "country_name", "country_name")
        region_freq = create_freq_dict(website_row, facebook_row, google_row, "main_region", "region_name", "region_name")
        google_row["new_phone"] = remove_plus_spaces_lines(google_row["phone"])
        facebook_row["new_phone"] = remove_plus_spaces_lines(facebook_row["phone"])
        website_row["new_phone"] = remove_plus_spaces_lines(website_row["phone"])
        phone_number_freq = create_freq_dict(website_row, facebook_row, google_row, "new_phone", "new_phone", "new_phone")
        name_freq = create_freq_dict(website_row, facebook_row, google_row, "site_name", "name", "name")
        
        new_row[1] = max(country_freq, key=country_freq.get) if len(country_freq) != 0 else ""
        new_row[2] = max(region_freq, key=region_freq.get) if len(region_freq) != 0 else ""
        new_row[3] = max(phone_number_freq, key=phone_number_freq.get) if len(phone_number_freq) != 0 else ""
        if google_row["new_phone"] == new_row[3]:
            new_row[3] = google_row["phone"]
        elif facebook_row["new_phone"] == new_row[3]:
            new_row[3] = facebook_row["phone"]
        elif website_row["new_phone"] == new_row[3]:
            new_row[3] = website_row["phone"]

        new_row[4] = max(name_freq, key=name_freq.get) if len(name_freq) != 0 else ""
        
        if pd.notna(google_row["name"]):
            if google_row["name"].lower() == new_row[4]:
                new_row[4] = google_row["name"]
            elif pd.notna(facebook_row["name"]):
                if facebook_row["name"].lower() == new_row[4]:
                    new_row[4] = facebook_row["phone"]
                elif pd.notna(website_row["site_name"]):
                    if website_row["site_name"] == new_row[4]:
                        new_row[4] = website_row["site_name"]
        

    new_df = pd.concat([pd.DataFrame([new_row], columns=new_df.columns), new_df], ignore_index=True)

new_df.to_csv("merged_datasets.csv")

As we finished with the most obvious ones with unique values in all of our tables (from the perspective of the domain's column), we will try to identify the ones which have common domain from the facebook and website dataset. we will make an order with most similar: phone number (after parsing it) and name (at least contain the letters from it)

In [52]:
def add_new_line(website_row, facebook_row, google_row, new_row, new_df, prioritize_website=False):
    if prioritize_website:
        if (pd.isna(google_row["domain"])) or validate_rows_are_similar(facebook_row, google_row, "name", "name", "phone", "phone") > 0:
            google_row = dummy_google_df.iloc[0]
            facebook_row = dummy_facebook_df.iloc[0]
    new_row[0] = str(facebook_row["categories"]) \
        if pd.notna(facebook_row["categories"]) else str(website_row["s_category"]) \
        if pd.notna(website_row["s_category"]) else str(google_row["category"])
    if pd.isna(website_row["s_category"]) and str(website_row["s_category"]) not in new_row[0]:
        new_row[0] += "|" + str(website_row["s_category"])
    if pd.notna(google_row["category"]) and str(google_row["category"]) not in new_row[0]:
        new_row[0] += "|" + str(google_row["category"])
    country_freq = create_freq_dict(website_row, facebook_row, google_row, "main_country", "country_name", "country_name")
    region_freq = create_freq_dict(website_row, facebook_row, google_row, "main_region", "region_name", "region_name")
    google_row["new_phone"] = remove_plus_spaces_lines(google_row["phone"]) if pd.notna(google_row["phone"]) else ""
    facebook_row["new_phone"] = remove_plus_spaces_lines(facebook_row["phone"]) if pd.notna(facebook_row["phone"]) else ""
    website_row["new_phone"] = remove_plus_spaces_lines(website_row["phone"]) if pd.notna(website_row["phone"]) else ""
    phone_number_freq = create_freq_dict(website_row, facebook_row, google_row, "new_phone", "new_phone", "new_phone")
    name_freq = create_freq_dict(website_row, facebook_row, google_row, "site_name", "name", "name")

    new_row[1] = max(country_freq, key=country_freq.get) if len(country_freq) != 0 else ""
    new_row[2] = max(region_freq, key=region_freq.get) if len(region_freq) != 0 else ""
    new_row[3] = max(phone_number_freq, key=phone_number_freq.get) if len(phone_number_freq) != 0 else ""

    if google_row["new_phone"] == new_row[3]:
        new_row[3] = google_row["phone"]
    elif facebook_row["new_phone"] == new_row[3]:
        new_row[3] = facebook_row["phone"]
    elif website_row["new_phone"] == new_row[3]:
        new_row[3] = website_row["phone"]

    new_row[4] = max(name_freq, key=name_freq.get) if len(name_freq) != 0 else ""
        
    if pd.notna(google_row["name"]):
        if google_row["name"].lower() == new_row[4]:
            new_row[4] = google_row["name"]
        elif pd.notna(facebook_row["name"]):
            if facebook_row["name"].lower() == new_row[4]:
                new_row[4] = facebook_row["phone"]
            elif pd.notna(website_row["site_name"]):
                if website_row["site_name"] == new_row[4]:
                    new_row[4] = website_row["site_name"]
    
    
    return pd.concat([pd.DataFrame([new_row], columns=new_df.columns), new_df], ignore_index=True)

def validate_google_rows(google_row, other_row, row_name, row_phone):
    lambda_func = lambda x: validate_rows_are_similar(x, other_row, "name", row_name, "phone", row_phone)
    validate_rows = google_row.apply(lambda_func, axis=1)
    if len(validate_rows[validate_rows == 2]) != 0:
        return google_row[validate_rows == 2].iloc[0]
    elif len(validate_rows[validate_rows == 2]) != 0:
        return google_row[validate_rows == 1].iloc[0]
    else:
        return dummy_google_df.iloc[0]

def validate_rows_are_similar(row1, row2, row1_name, row2_name, row1_phone, row2_phone):
    ok = 0
    if pd.notna(row1[row1_phone]) and pd.notna(row2[row2_phone]):
        if remove_plus_spaces_lines(row1[row1_phone]) == remove_plus_spaces_lines(row2[row2_phone]):
            ok = 1
    if pd.notna(row1[row1_name]) and pd.notna(row2[row2_name]):
        if row1[row1_name] in row2[row2_name] or row2[row2_name] in row1[row1_name]:
            ok = ok + 1

    return ok

new_new_df = new_df.copy()

for el in duplicated_dom:
    website_row = website_df[website_df["root_domain"] == str(el)]
    facebook_row = facebook_df[facebook_df["domain"] == str(el)]
    google_row = google_df[google_df["domain"] == str(el)]
    if len(website_row) == 0:
        website_row = dummy_website_df
    if len(google_row) == 0:
        google_row = dummy_google_df
    new_row = [""] * 5 + [str(el)]

    facebook_row = facebook_row.iloc[0]
    website_row = website_row.iloc[0]
    # having a public domain, it is possible that the website and facebook actually have different companies
    # we will check them by a fairly simple assumption
    if validate_rows_are_similar(facebook_row, website_row, "name", "site_name", "phone", "phone") > 0:
        google_row = validate_google_rows(google_row, facebook_row, "name", "phone")
        new_new_df = add_new_line(website_row, facebook_row, google_row, new_row, new_new_df)
    else:
        google_row_one = validate_google_rows(google_row, facebook_row, "name", "phone")
        new_new_df = add_new_line(website_row, facebook_row, google_row_one, new_row, new_new_df)
        google_row_two = validate_google_rows(google_row, website_row, "site_name", "phone")
        new_new_df = add_new_line(website_row, facebook_row, google_row_two, new_row, new_new_df,True)

Now, we can save it (again)

In [53]:
new_new_df.to_csv("new_merged_datasets.csv")

Now, we can say that we finished with two datasets to join: website (except the 13 rows with corrupted domain) and facebook. Now, the only thing to do is to take the google dataset and put all of the others which were not used in the final dataset.

In [46]:
new_new_df = new_df.copy()
for el in duplicated_dom:
    
    website_row = website_df[website_df["root_domain"] == str(el)]
    website_row = website_row.iloc[0] if len(website_row) != 0 else dummy_website_df.iloc[0]
    facebook_row = facebook_df[facebook_df["domain"] == str(el)].iloc[0]
    google_row = google_df[google_df["domain"] == str(el)]
    
    lambda_facebook_func = lambda x: validate_rows_are_similar(x, facebook_row, "name", "name", "phone", "phone")
    lambda_website_func = lambda x: validate_rows_are_similar(x, website_row, "name", "site_name", "phone", "phone")

    validate_facebook_rows = google_row.apply(lambda_facebook_func, axis=1)
    validate_website_rows = google_row.apply(lambda_website_func, axis=1)
    
    no_common_facebook = validate_facebook_rows[validate_facebook_rows == 0].index.tolist()
    no_common_website = validate_website_rows[validate_website_rows == 0].index.tolist()
    merged_list = list(set(no_common_facebook + no_common_website))
    for i in merged_list:
        add_google_row = google_df.iloc[i]
        # this line does not have anything in common with the other two datasets, so we can add it immediately
        new_row = [
            add_google_row["category"],
            add_google_row["country_name"],
            add_google_row["region_name"],
            add_google_row["phone"],
            add_google_row["name"],
            add_google_row["domain"]
        ]
        
        new_new_df = pd.concat([pd.DataFrame([new_row], columns=new_new_df.columns), new_new_df], ignore_index=True)

Now save it again, again and again... (we save it so often in case we lose the jupyter session)

In [47]:
new_new_df.to_csv("new_new_merged_datasets.csv")

Now, for having a final dataset/csv with only the columns wanted, we will delete domain column and save it again as "final_merged_dataset.csv".

In [48]:
final_df = new_new_df.drop(columns=["Domain"])
final_df.to_csv("final_merged_dataset.csv")

Conclusions:

1. in this notebook, we analysed  and merged three datasets: facebook, google and website csvs.
2. we observed that: 
    - the names of the companies from the three datasets can differ with a word or two
    - more companies can have the same phone number (probably because they have the same owner) and also the same names
    - domain column is the only one which has unique elements in two datasets: "facebook" and "website", and also all of the elements from the other tables are non-null
    - each element from facebook domain column can be found in the other two datasets, and vice-versa (except 13 elements from "website").
    - 15000 of the elements are not unique for google dataset, and can be found at different companies, but nonetheless, all of them exists in the other tables (except "website", from those 13 elements) 
    - altough domain is a really good start, in the "website" table there are 13 data which seem to have really weird domain, which will be ignored
3. The steps for joining all of the tables are:
    1. Getting all of the domain elements from facebook (which are already unique)
    2. Separating them into unique and duplicated from the perspective of google dataset
    3. Solving the ones which are unique by taking all the elements from all the tables with the same domain, and create a majority vote for getting a name, phone number, country or region. If the vote is equal, the first one will be taken.
    4. After that, we take the duplicated elements from google dataset and we will put the companies from facebook and website dataset, in the same manner, but adding a similiarity vote, which verify if the other company has at least a phone number or the name in common with the other, just to be sure that we do not put two times the same company and also for finding out which company from the google csv could be from the facebook csv or website csv
    5. At last, we take the rest of the companies from google (which were not used and do not have a pair with the other datasets), and put them one by one in the new dataset.